# Ayudantía 8: Webservices 🌐

## Ayudantes  👾

- [Clemente Campos](https://github.com/mskdancers)
- [Patricio Hinostroza](https://github.com/Dvckhv)
- [Julio Huerta](https://github.com/julius)
- [Carlos Olguin](https://github.com/CarlangaUC)
- [Catalina Miranda](https://github.com/catalinamirandah)
- [Felipe Vidal](https://github.com/fvidalf)

## ¿Qué es un *webservice*?

Para mejor entendimiento, *webservice* hace referencia al **conjunto de protocolos y estándares que permiten la comunicación cliente-servidor mediante la web**, es decir, las aplicaciones que permiten comunicar diversas máquinas mediante la web.

## ¿Qué es una API?

Una API (*Application Programming Interface*), es vista como el **conjunto de funciones ofrecidas** (es decir, visibles) por un servicio para ser usadas por otros programas. 

Haciendo el símil con *networking*, desde el punto de vista del cliente, el servidor al que nos intentamos conectar contendría una API, pues este recibe peticiones y posee funciones destinadas a entregarnos resultados como respuesta. Más gráficamente...

<img width=800 src="img/example.png"> <br>
<sup>Para mas informacion: https://medium.com/geekculture/a-beginners-guide-to-apis-9aa7b1b2e172</sup>


## HTTP

En este curso utilizaremos (o consumiremos) *APIs* y *Webservices* en Python mediante la libreria **requests**, que requiere el uso del protocolo HTTP. Este contiene diversos métodos/códigos para emplear el patrón **request-response** (solicitudes-respuestas), siendo el cliente el cual realiza solicitudes al servidor y este entrega una respuesta. Un ejemplo de lo planteado sería:

In [1]:
import requests

url = "https://github.com/IIC2233/Syllabus"
response = requests.get(url)
print(f'Solicitud realizada con status: {response.status_code}')

url_erronea = "https://github.com/IIC2233/Penca!"
response = requests.get(url_erronea)
print(f'Solicitud realizada con status: {response.status_code}')


Solicitud realizada con status: 200
Solicitud realizada con status: 404


En este caso notamos que para la primera consulta nos arrojó el código **200** significando que fue realizada correctamente, mientras que la segunda consulta fue con **404** significando que no se encontró el contenido solicitado.

## Métodos de consulta

HTTP define métodos (caracterizados por un verbo) para ejecutar varios tipos de consulta, con diferentes objetivos. Los principales son los siguientes:

### GET

Recupera u obtiene un recurso en el servidor. No hace cambios.

In [2]:
url = "http://www.boredapi.com/api/activity"
response_get = requests.get(url)

print(response_get.json())

{'activity': 'Go on a long drive with no music', 'type': 'relaxation', 'participants': 1, 'price': 0.1, 'link': '', 'key': '4290333', 'accessibility': 0.2}


### POST

Crea un recurso en el servidor.

In [3]:
url = "https://jsonplaceholder.typicode.com/posts"
data = {
    "title": "Plan súper original para pasar el rato",
    "body": response_get.json()["activity"],
    "link": response_get.json()["link"]
}
response_post = requests.post(url, data=data)

print(response_post.json())

{'title': 'Plan súper original para pasar el rato', 'body': 'Go on a long drive with no music', 'link': '', 'id': 101}


### PATCH

Actualiza parcialmente un recurso existente en el servidor.

In [4]:
url = "https://pythonexamples.org/"
data = {
    "some data": "just an example",
    "more data": "literally anything else"
}

response_patch = requests.patch(url, data=data)

print(response_patch.status_code)

200


### DELETE

Elimina un recurso del servidor.

In [8]:
url = "https://jsonplaceholder.typicode.com/photos/"

# Supongamos que borraremos algún recurso con id = 1
_id = 1

response_delete = requests.delete(url + f"{_id}")

print(response_delete.status_code)

200


En algunos casos, vamos a necesitar ciertos permisos para poder crear, actualizar o borrar recursos de algún servidor. En el siguiente ejemplo vamos a ver cómo podemos manejar esto.

## Ejemplo: API de github

En este ejemplo vamos a crear una issue en nuestro repositorio de github usando la API de github. Para hacer esto le entregaremos un diccionario mediante el parámetro `data`, que contendrá el título y el cuerpo de nuestra issue. Además, vamos a necesitar un token, que podemos generar en [esta página](https://github.com/settings/tokens/new). Este token debe tener al menos el siguiente permiso: _Full control of private repositories_, y se lo enviaremos a la API con el parámetro `headers`.

In [ ]:
import json, requests

github_repo = ''
token = ""

body = {
    # Completar con lo que quieran!
    'title': "",
    'body': ""
}

my_headers = {
    'Authorization': 'token ' + token,
    'Accept': 'application/vnd.github.v3+json'
}

url = f"https://api.github.com/repos/IIC2233/{github_repo}/issues"

response = requests.post(url, data=json.dumps(body), headers=my_headers)
print(response.status_code)

## APInterests

¡Estamos desarrollando una aplicación para conectar personas, Interests! Para eso, tenemos nuestra propia base de datos (una lista) que mediante complejos algoritmos (un filtro) permite que un usuario conozca a otras personas con intereses similares, sean pasatiempos, gustos, actividades, ¡lo que sea que quieran compartir!

Por ahora estamos definiendo las funcionalidades de la API, y necesitamos que alguien las pruebe para ver que todo esté en orden. Para eso, están disponibles los siguientes *endpoints*:

### Endpoints disponibles

El servidor implementado en server.py cuenta con:

#### GET: /users

Retorna todos los usuarios registrados mediante el campo "users" de la respuesta json.

#### GET: /user/{username}

Retorna un único usuario dado el username proporcionado en el path. Este es retornado mediante el campo "user" de la respuesta json.
En caso de no encontrarse, se retornará un *status_code* 404, y en la respuesta json, el campo "message" será "Usuario no encontrado :("

#### POST: /register

Registra un usuario en la base de datos. Como parámetro `json` debe recibir los campos propios del usuario en forma de diccionario. Esto es:
```
{
    "nombre": <nombre>,
    "username": <username>
    "edad": <edad>,
    "intereses": [<interes_1>, <interes_2>, ..., <interes_n>],
    "dato_freak": <dato_freak>
}
```

En caso de realizarse exitosamente, el usuario estará disponible mediante el campo "user" de la respuesta json. Además, **se le asignará un id único**. En caso contrario, se retornará un *status_code* 400 y el campo "message" de la respuesta json será "Error al registrar usuario".

#### GET: /group/{interes}

Retorna un grupo de a lo más 3 usuarios mediante el campo "users" de la respuesta json. Los usuarios comparten el interés entregado en el path. En caso de no existir ningún usuario con ese interés, retornará un *status code* 404 y el campo "message" será "No hay usuarios registrados con ese interés :(".

#### PATCH: /user/{id}

Actualiza el campo "intereses" de un usuario dado mediante el id entregado en el path. La nueva lista de intereses se debe recibir mediante el parámetro json de la siguiente forma:
```
{
    "intereses": [<interes_1>, <interes_2>, ..., <interes_n>]
}
```

### Lo que debes implementar

En `client.py` te encontrarás con un flujo de programa que muestra un menú sencillo por consola. Deberás completar cada una de las opciones, asociadas a las siguientes tareas:

#### 1. Obtener todos los usuarios
Se debe realizar la request correspondiente, e imprimir cada usuario recibido por la API a la consola.

#### 2. Registrar un usuario
Se debe recibir por consola un *str* 'nombre', un *str* 'username', un *int* 'edad', un *str* 'intereses' y un *str* 'dato_freak'. El *str* 'intereses' deberá corresponder a valores separables por comas, por ejemplo "jardineria,pasear por las tardes,tomar cafe". 

Luego, se debe realizar la request correspondiente. En caso de que esta sea exitosa, se debe imprimir el usuario guardado (retornado por la API).

#### 3. Obtener un grupo aleatorio por interés
Se debe recibir por consola un *str* 'interes'. Luego, se debe realizar la request correspondiente. En caso de que sea exitosa, se deberá imprimir cada uno de los usuario retornados por la API. En caso contrario, se deberá imprimir el *status code* resultante.

#### 4. Actualizar los intereses de un usuario dado
Se debe recibir por consola un *str* 'username' y un *str* 'intereses' del mismo formato que el usado al registrar. Luego, se debe realizar la request correspondiente. En caso de que esta sea exitosa, se debe imprimir el usuario actualizado (retornado por la API).

#### 5. Registrar todos los usuarios en users.csv
Se debe leer y procesar el archivo users.csv, para luego realizar las requests correspondientes para registrar a cada usuario.


**Además**, para toda request deberás imprimir el campo "message" de la respuesta json, independientemente de si esta fue exitosa o no.

Para probar tus requests, puedes ejecutar el archivo server.py (también en la carpeta APInterests) en una terminal, y desde otra, ejecutar client.py